In [ ]:
import pandas as pd
import tensorflow as tf
from IPython.display import clear_output

In [ ]:
dftrain = pd.read_csv("datasets/diabetes-dataset-females/diabetes.csv")
y_train = dftrain.pop("Outcome")

dfeval = dftrain
y_eval = y_train

dfeval

In [ ]:
feature_columns = []
CATEGORICAL_COLUMNS = []
NUMERICAL_COLUMNS = ["Pregnancies",	"Glucose",	"BloodPressure",
                     "SkinThickness",	"Insulin",	"BMI",	"DiabetesPedigreeFunction",	"Age"]
for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
for feature_name in NUMERICAL_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

feature_columns

In [ ]:
def make_input_fn(data_df, label_df, shuffle=True, num_epochs=10, batch_size=32):
    def input_fn():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_fn

In [ ]:
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, True, 1)
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
results = linear_est.evaluate(eval_input_fn)
# clear_output()
results["accuracy"]

In [ ]:
# tmpdir = tempfile.mkdtemp()

serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns=feature_columns))

# estimator_base_path = os.path.join(tmpdir, 'from_estimator')
estimator_path = linear_est.export_saved_model("../tf2/", serving_input_fn)